In [36]:
%load_ext autoreload
%autoreload 2
from evolution_algos import cem_uncorrelated, saes_1_1, ObjectiveFunction
from evolution_policy import NeuralNetworkPolicy, LogisticRegression

import numpy as np
from test_utils import RenderWrapper
from flatland.envs.line_generators import SparseLineGen
from flatland.envs.malfunction_generators import (
    MalfunctionParameters,
    ParamMalfunctionGen,
)
from flatland.envs.persistence import RailEnvPersister
from flatland.envs.rail_env import RailEnv
from flatland.envs.rail_generators import SparseRailGen
# from flatland_cutils import TreeObsForRailEnv as TreeCutils
from flatland.envs.observations import GlobalObsForRailEnv

from flatland.envs.observations import TreeObsForRailEnv
from flatland.envs.distance_map import DistanceMap
import flatland.envs.rail_env_shortest_paths as sp

from flatland.utils.rendertools import RenderTool

from observation_utils import normalize_observation

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
# Create the environment
observation_tree_depth = 2
observation_radius = 10
observation_max_path_depth = 20

env = RailEnv(
    width=20,
    height=15,
    rail_generator=SparseRailGen(
        seed=0,  # Random seed
        max_num_cities=2,  # Number of cities
        grid_mode=True,
        max_rails_between_cities=2,
        max_rail_pairs_in_city=1,
    ),
    line_generator=SparseLineGen(speed_ratio_map={1.: 1.}
        ),
    number_of_agents=2, # Only one agent
    obs_builder_object=TreeObsForRailEnv(max_depth=observation_tree_depth),
    malfunction_generator=ParamMalfunctionGen(
        MalfunctionParameters(
            malfunction_rate=0.,  # Rate of malfunction
            min_duration=3,  # Minimal duration
            max_duration=20,  # Max duration
        )
    ),
)

In [42]:
# Initializing policy
n_features_per_node = env.obs_builder.observation_dim
n_nodes = sum([np.power(4, i) for i in range(observation_tree_depth + 1)])
state_size = n_features_per_node * n_nodes
policy = LogisticRegression(state_size, 5)
print(policy.num_params)

1155


In [43]:
theta = np.random.rand(policy.num_params)

obs, info = env.reset()
agent_obs = [None] * env.get_num_agents()
for agent in env.get_agent_handles():
    if obs[agent]:
        agent_obs[agent] = normalize_observation(obs[agent], tree_depth=observation_tree_depth, observation_radius=observation_radius)

for agent in env.get_agent_handles():
    action = policy.act(agent_obs[agent], theta)

len(theta)
    

In [44]:
# Cross entropy method

# Create the objective function
objective_function = ObjectiveFunction(env, policy,observation_tree_depth=observation_tree_depth, observation_radius=observation_radius, num_episodes=1 , max_time_steps=200)

init_mean_array = np.random.random(size = policy.num_params)
init_var_array = np.ones(shape=policy.num_params) * 100.
hist_dict = {}
theta = cem_uncorrelated(
    objective_function,
    mean_array = init_mean_array,
    var_array = init_var_array,
    max_iterations=50,
    sample_size=50,
    elite_frac=0.2,
    print_every=1,
    success_score=15,
    hist_dict=hist_dict
)

iteration :  0
sorted samples :  [-0.0, -0.0, 20.0, 21.0, 22.0, 23.0, 24.0, 24.0, 25.0, 29.0, 30.0, 31.0, 32.0, 35.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0, 39.0, 40.0, 40.0, 40.0, 41.0, 41.0, 41.0, 42.0, 43.0, 43.0, 43.0, 43.0, 43.0, 46.0, 46.0, 46.0, 46.0, 46.0, 46.0, 46.0, 46.0, 46.0, 46.0, 52.0, 53.0, 54.0, 57.0, 58.0]
37.86
iteration :  1
sorted samples :  [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 2.0, 12.0, 14.0, 20.0, 21.0, 21.0, 21.0, 21.0, 22.0, 22.0, 22.0, 22.0, 23.0, 24.0, 25.0, 27.0, 29.0, 30.0, 35.0, 38.0, 38.0, 38.0, 39.0, 39.0, 39.0, 40.0, 40.0, 41.0, 41.0, 42.0, 42.0, 42.0, 43.0, 44.0, 45.0, 46.0, 46.0, 46.0, 46.0, 46.0, 53.0]
26.94
iteration :  2
sorted samples :  [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 19.0, 24.0, 24.0, 27.0, 27.0, 27.0, 28.0, 28.0, 29.0, 30.0, 30.0, 30.0, 30.0, 30.0, 31.0, 31.0, 39.0, 40.0, 40.0, 41.0, 41.0, 41.0, 41.0, 41.0, 43.0, 43.0, 43.0, 43.0, 43.0, 44.0, 44.0,

In [45]:
len(theta)

1155

In [46]:
# Create the environment
observation_tree_depth = 3
observation_radius = 10
observation_max_path_depth = 20

env = RailEnv(
    width=20,
    height=15,
    rail_generator=SparseRailGen(
        seed=0,  # Random seed
        max_num_cities=2,  # Number of cities
        grid_mode=True,
        max_rails_between_cities=2,
        max_rail_pairs_in_city=1,
    ),
    line_generator=SparseLineGen(speed_ratio_map={1.: 1.}
        ),
    number_of_agents=2, # Only one agent
    obs_builder_object=TreeObsForRailEnv(max_depth=observation_tree_depth),
    malfunction_generator=ParamMalfunctionGen(
        MalfunctionParameters(
            malfunction_rate=0.,  # Rate of malfunction
            min_duration=3,  # Minimal duration
            max_duration=20,  # Max duration
        )
    ),
)

In [47]:
# Initializing policy
n_features_per_node = env.obs_builder.observation_dim
n_nodes = sum([np.power(4, i) for i in range(observation_tree_depth + 1)])
state_size = n_features_per_node * n_nodes
policy = LogisticRegression(state_size, 5)
print(policy.num_params)

4675


In [48]:
# Cross entropy method

# Create the objective function
objective_function = ObjectiveFunction(env, policy,observation_tree_depth=observation_tree_depth, observation_radius=observation_radius, num_episodes=1 , max_time_steps=200)

init_mean_array = np.random.random(size = policy.num_params)
init_var_array = np.ones(shape=policy.num_params) * 100.
hist_dict = {}
theta = cem_uncorrelated(
    objective_function,
    mean_array = init_mean_array,
    var_array = init_var_array,
    max_iterations=50,
    sample_size=50,
    elite_frac=0.2,
    print_every=1,
    success_score=10,
    hist_dict=hist_dict
)

iteration :  0
sorted samples :  [-0.0, 3.0, 13.0, 21.0, 21.0, 21.0, 23.0, 26.0, 27.0, 28.0, 28.0, 29.0, 29.0, 30.0, 30.0, 34.0, 36.0, 37.0, 37.0, 37.0, 37.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0, 40.0, 41.0, 43.0, 44.0, 45.0, 46.0, 46.0, 46.0, 46.0, 46.0, 46.0, 46.0, 46.0, 55.0, 55.0, 57.0]
35.78
iteration :  1
sorted samples :  [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 9.0, 16.0, 22.0, 22.0, 22.0, 24.0, 24.0, 25.0, 26.0, 27.0, 27.0, 27.0, 27.0, 28.0, 29.0, 30.0, 30.0, 30.0, 30.0, 35.0, 36.0, 37.0, 38.0, 38.0, 38.0, 38.0, 39.0, 40.0, 41.0, 41.0, 42.0, 42.0, 43.0, 43.0, 44.0, 44.0, 45.0, 46.0, 46.0, 46.0, 46.0, 46.0, 50.0, 51.0]
30.6
iteration :  2
sorted samples :  [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 2.0, 6.0, 12.0, 19.0, 20.0, 21.0, 21.0, 24.0, 26.0, 27.0, 28.0, 29.0, 30.0, 30.0, 30.0, 30.0, 30.0, 31.0, 35.0, 39.0, 39.0, 41.0, 42.0, 42.0, 42.0, 43.0, 43.0, 44.0, 44.0, 44.0, 45.0, 45.0, 45.0, 45.

In [35]:
len(theta)

275